# **OpenAI Interaction**

![OpenAI](assets/openai-logos-2025/OpenAI-logos(new)/SVGs/OpenAI-black-monoblossom.svg)

## Python Requests Library

In [58]:
import os
import requests
import json
from lib.ws_minify import ws_minify
from lib.get_random_ollama_model import get_random_ollama_model
import random

# Compose a user prompt. `textwrap.dedent` removes the common leading indentation,
# which keeps the code readable without sending leading spaces to the model.
user_prompt = ws_minify("""
    State your model and the company or lab that created you.
    Then discuss a little bit about why you were created and what are your strengths and weaknesses.
    Finally discuss situations in which it would be preferable to choose you over other models.
""")

# Endpoint for listing models
url = "https://api.openai.com/v1/models"
response = requests.get(url, headers=headers, timeout=30)


headers = {
    "Authorization": f"Bearer {os.environ["OPENAI_API_KEY"]}",
    "Content-Type": "application/json",
}

openai_models = [openai_model["id"] for openai_model in response.json().get("data", [])]

def is_chat_model(model_id: str) -> bool:
    """Heuristic filter: include chat-capable LLMs; exclude embeddings/moderation/audio."""
    s = model_id.lower()
    if any(bad in s for bad in ("embedding", "embed", "moderation", "whisper", "tts", "audio", "transcribe", "pro", "turbo", "search", "preview")):
        return False
    # Common chat families (Responses-friendly): GPT-x and o-series
    return s.startswith(("gpt-", "gpt", "o", "chatgpt"))

chat_models = sorted([m for m in openai_models if is_chat_model(m)])

current_model = random.choice(chat_models)

# Endpoint for generating text
url = "https://api.openai.com/v1/responses"

payload = {
    "model": current_model,
    "input": user_prompt
}

response = requests.post(url, headers=headers, json=payload, timeout=60)
response_json = response.json()

def extract_text(response: dict) -> str:
    # Prefer the convenience field if present; otherwise collect text from parts
    if "output_text" in response and response["output_text"]:
        return response["output_text"]
    parts = []
    for item in response.get("output", []):
        for c in item.get("content", []):
            if isinstance(c, dict) and "text" in c:
                parts.append(c["text"])
    return "".join(parts)

# Print which model we used for clarity.
print(f"{current_model}\n")

print(extract_text(response_json))

# Simple visual separator for readability.
print("\n------------------------------------------")
print("\n\n")


o3-2025-04-16

Model  
• Name: ChatGPT (based on OpenAI’s GPT-4 architecture)  
• Creator: OpenAI, an AI research and deployment company  

Why the model was created  
OpenAI’s mission is to ensure that artificial general intelligence benefits all of humanity. GPT-4 was trained to:  
1. Serve as a general-purpose natural-language interface, enabling people to obtain information, write, code, brainstorm, translate, tutor and more through ordinary conversation.  
2. Demonstrate and advance the state of large-language-model research, providing insight into alignment, safety and responsible deployment at scale.  
3. Offer developers and organizations a flexible tool they can adapt for specialized applications (e.g., customer support, education, creative tools, programming assistants).

Core strengths  
• Breadth of knowledge: Trained on a large and diverse text corpus, it can address a wide range of topics.  
• Reasoning ability: Performs well on many standardized tests and multi-step prob

## OpenAI Python SDK Library

In [64]:
import os
from openai import OpenAI
import json
from lib.ws_minify import ws_minify
from lib.get_random_ollama_model import get_random_ollama_model
import random

# Initialize the SDK client
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Compose a user prompt. `textwrap.dedent` removes the common leading indentation,
# which keeps the code readable without sending leading spaces to the model.
user_prompt = ws_minify("""
    State your model and the company or lab that created you.
    Then discuss a little bit about why you were created and what are your strengths and weaknesses.
    Finally discuss situations in which it would be preferable to choose you over other models.
""")

all_models = client.models.list()
openai_models = [model.id for model in all_models.data]

def is_chat_model(model_id: str) -> bool:
    """Heuristic filter: include chat-capable LLMs; exclude embeddings/moderation/audio."""
    s = model_id.lower()
    if any(bad in s for bad in (
        "embedding", "embed", "moderation", "whisper",
        "tts", "audio", "transcribe", "pro", "turbo",
        "search", "preview"
    )):
        return False
    return s.startswith(("gpt-", "gpt", "o", "chatgpt"))

chat_models = sorted([m for m in openai_models if is_chat_model(m)])
current_model = random.choice(chat_models)

def extract_text(response) -> str:
    if getattr(response, "output_text", None):
        return response.output_text
    # fallback: collect pieces
    parts = []
    for item in response.output or []:
        for c in getattr(item, "content", []):
            if hasattr(c, "text"):
                parts.append(c.text)
    return "".join(parts)

response = client.responses.create(
    model=current_model,
    input=user_prompt,
)


# Print which model we used for clarity.
print(f"{current_model}\n")

# The generated text
print(extract_text(response))

# Simple visual separator for readability.
print("\n------------------------------------------")
print("\n\n")


gpt-4.1-mini-2025-04-14

I am ChatGPT, based on the GPT-4 architecture developed by OpenAI. OpenAI is an AI research lab focused on ensuring that artificial general intelligence benefits all of humanity.

**Why I was created:**  
I was designed to assist with a wide range of natural language tasks, such as answering questions, generating text, helping with creative projects, providing explanations, and supporting learning. The goal is to make complex information accessible, improve productivity, and enable new forms of human-computer interaction.

**Strengths:**  
- I can generate coherent and contextually relevant text across many topics.  
- I have a broad knowledge base up to my last training cutoff (June 2024).  
- I am capable of nuanced understanding and can handle creative and technical prompts alike.  
- I support image input capabilities, allowing me to analyze and discuss visual content.  
- I can adapt my tone and style for different audiences and purposes.

**Weaknesses:** 